In [60]:
import vectorbt as vbt
import numpy as np
import numpy.ma as ma
import pandas as pd
import inspect
import datetime
import talib
from IPython.display import display, HTML


vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [61]:
symbol = 'qqq'
data = vbt.AlpacaData.download(symbol,
                               start='2022-1-01',
                               end='2023-3-16',
                               limit=2000000,
                               timeframe='5m').get()

inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


FigureWidget({
    'data': [{'close': array([395.98, 395.97, 395.88, ..., 298.69, 298.59, 298.77]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([395.98, 395.97, 395.88, ..., 298.99, 298.68, 298.79]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([395.98, 395.97, 395.52, ..., 298.53, 298.52, 298.58]),
              'name': 'Candlestick',
              'open': array([395.98, 395.97, 395.52, ..., 298.99, 298.68, 298.58]),
              'type': 'candlestick',
              'uid': 'dae4dd33-2857-4010-8411-76d1a90a7ff4',
              'x': array([datetime.datetime(2022, 1, 3, 5, 50, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 1, 3, 8, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2022, 1, 3, 9, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 3, 15, 23, 45, tzinfo=datetime.timezone.utc),
            

In [62]:
bullish_engulfing = talib.CDLENGULFING(data['Open'], data['High'], data['Low'], data['Close'])

print(bullish_engulfing[bullish_engulfing == 100])

timestamp
2022-01-03 13:45:00+00:00    100
2022-01-03 14:15:00+00:00    100
2022-01-03 14:30:00+00:00    100
2022-01-03 16:20:00+00:00    100
2022-01-03 21:35:00+00:00    100
                            ... 
2023-03-15 17:05:00+00:00    100
2023-03-15 17:50:00+00:00    100
2023-03-15 18:50:00+00:00    100
2023-03-15 19:55:00+00:00    100
2023-03-15 23:55:00+00:00    100
Length: 2063, dtype: int32


In [63]:
RSI = vbt.IndicatorFactory.from_talib('RSI')
SMA = vbt.IndicatorFactory.from_talib('SMA')


def three_ma(start, close, high, low, rsi_window = 14, sma_window_large = 200, sma_window_medium = 50, sma_window_small = 21, rsi_buy = 50):
    rsi = RSI.run(close, rsi_window).real.to_numpy()
    sma_large = SMA.run(close, sma_window_large).real.to_numpy()
    sma_medium = SMA.run(close, sma_window_medium).real.to_numpy()
    sma_small = SMA.run(close, sma_window_small).real.to_numpy()

    buy = np.where((sma_small>sma_medium) & (sma_medium>sma_large) & (bullish_engulfing==100) & (rsi > rsi_buy), 1, 0)

    stop_loss = -1*(((low - (high-low)*2)-close)/close)

    profit = ((high + ((high-low)*4)-close)/close)

    return buy, profit, stop_loss


three_ma = vbt.IndicatorFactory(
    class_name = "three_ma",
    short_name = "three_ma",
    input_names = ['start', 'close', 'high', 'low'],
    output_names = ['buy', 'profit', 'stop_loss'],
    param_names=['rsi_window', 'sma_window_large', 'sma_window_medium', 'sma_window_small', 'rsi_buy']
).from_apply_func(three_ma, rsi_window = 14, sma_window_large = 200, sma_window_medium = 50, sma_window_small = 21, rsi_buy = 50)

order = three_ma.run(
    data['Open'],
    data['Close'],
    data['High'],
    data['Low'],
    rsi_window = 14,
    sma_window_large = 200,
    sma_window_medium = 50,
    sma_window_small = 21,
    rsi_buy = 50,
    param_product = True,
    to_2d = False
)

entries = order.buy == 1

pf = vbt.Portfolio.from_signals(data['Open'], entries, sl_stop = order.stop_loss, tp_stop = order.profit, init_cash=10000, freq='5m')

#orders = pf.orders
pf.stats()




Start                         2022-01-03 05:50:00+00:00
End                           2023-03-15 23:55:00+00:00
Period                                206 days 15:30:00
Start Value                                     10000.0
End Value                                  13388.836754
Total Return [%]                              33.888368
Benchmark Return [%]                         -24.597202
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               8.953087
Max Drawdown Duration                  49 days 19:10:00
Total Trades                                        255
Total Closed Trades                                 255
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  52.941176
Best Trade [%]                                 3.318893
Worst Trade [%]                               -3